# Assignment 2: Building a Simple Search Index

In this assignment, we will build a simple search index, which we will use later for Boolean retrieval. The assignment tasks are again at the bottom of this document.

## Loading the Data

In [1]:
Summaries_file = 'data/tree_Summaries.pkl.bz2'
Abstracts_file = 'data/tree_Abstracts.pkl.bz2'

In [2]:
import pickle, bz2
from collections import namedtuple

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )
    
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

Let's have a look at what the data looks like for our example paper:

In [10]:
Summaries[29420476]

paper(title='Evolutionary history of the angiosperm flora of China.', authors=['Lu LM', 'Mao LF', 'Yang T', 'Ye JF', 'Liu B', 'Li HL', 'Sun M', 'Miller JT', 'Mathews S', 'Hu HH', 'Niu YT', 'Peng DX', 'Chen YH', 'Smith SA', 'Chen M', 'Xiang KL', 'Le CT', 'Dang VC', 'Lu AM', 'Soltis PS', 'Soltis DE', 'Li JH', 'Chen ZD'], year=2018, doi='10.1038/nature25485')

In [11]:
Abstracts[29420476]

"High species diversity may result from recent rapid speciation in a 'cradle' and/or the gradual accumulation and preservation of species over time in a 'museum'. China harbours nearly 10% of angiosperm species worldwide and has long been considered as both a museum, owing to the presence of many species with hypothesized ancient origins, and a cradle, as many lineages have originated as recent topographic changes and climatic shifts-such as the formation of the Qinghai-Tibetan Plateau and the development of the monsoon-provided new habitats that promoted remarkable radiation. However, no detailed phylogenetic study has addressed when and how the major components of the Chinese angiosperm flora assembled to form the present-day vegetation. Here we investigate the spatio-temporal divergence patterns of the Chinese flora using a dated phylogeny of 92% of the angiosperm genera for the region, a nearly complete species-level tree comprising 26,978 species and detailed spatial distribution 

## Some Utility Functions

We'll define some utility functions that allow us to tokenize a string into terms, perform linguistic preprocessing on a list of terms, as well as a function to display information about a paper in a nice way. Note that these tokenization and preprocessing functions are rather naive - you may have to make them smarter in a later assignment.

In [7]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

print(preprocess(tokenize("Lorem ipsum dolor sit AMET")))

['lorem', 'ipsum', 'dolor', 'sit', 'amet']


In [9]:
from IPython.display import display, HTML
import re

def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/%s>%s</a>' % (s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>%s</em></small>' % Abstracts[id])
    if (show_id):
        lines.append('[ID: %d]' % id)
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

display_summary(29420476)
display_summary(29420476, show_abstract=True)

## Creating our first index

We will now create an _inverted index_ based on the words in the abstracts of the papers in our dataset.

We will implement our inverted index as a **Python dictionary with terms as keys and posting lists as values**. For the posting lists, instead of using Python lists and then implementing the different operations on them ourselves, we will use **Python sets** and use the predefined set operations to process these posting "lists". This will also ensure that each document is added at most once per term. The use of Python sets is not the most efficient solution but will work for our purposes. (As an optional additional exercise, you can try to implement the posting lists as Python lists for this and the following assignments.)

Not every paper in our dataset has an abstract; we will only index papers for which an abstract is available.

In [12]:
from collections import defaultdict

inverted_index = defaultdict(set)

# This may take a while:
for (id, abstract) in Abstracts.items():
    for term in preprocess(tokenize(abstract)):
        inverted_index[term].add(id)

Let's see what's in the index for the example term 'madagascar':

In [14]:
print(inverted_index['madagascar'])

{5644801, 27394434, 17894402, 18077188, 27084294, 26860934, 25794184, 18058249, 12118407, 18424844, 28943886, 22611855, 12736783, 26044948, 26398231, 15975831, 28081689, 11458587, 114465, 11454629, 28547194, 12682922, 28556334, 11336751, 24661555, 23544118, 22937148, 15579390, 11729086, 24294082, 25809223, 22694729, 21989710, 26236886, 24228694, 27664344, 19773273, 23180122, 22019930, 27848158, 26360928, 18452578, 6772964, 21437928, 15019625, 17646954, 18280171, 26760300, 14653803, 21044207, 23495153, 27928691, 15619444, 28380021, 20079861, 23825658, 26377339, 23345276, 15977086, 10601983}


We can now use this inverted index to answer simple one-word queries, for example to show all papers that contain the word 'amsterdam':

In [16]:
query_word = 'amsterdam'
for i in inverted_index[query_word]:
    display_summary(i)

----------

# Tasks

**Your name:** ...

### Task 1

Construct a function called `and_query` that takes as input a single string, consisting of one or more words, and returns a list of matching documents. `and_query`, as its name suggests, should require that all query terms are present in the documents of the result list. Demonstrate the working of your function with an example (choose one that leads to fewer than 100 hits to not overblow this notebook file).

(You can use the `tokenize` and `preprocess` functions we defined above to tokenize and preprocess your query. You can also exploit the fact that the posting lists are [sets](https://docs.python.org/3/library/stdtypes.html#set), which means you can easily perform set operations such as union, difference and intersect on them.)

In [ ]:
# Add your code here

### Task 2

Construct a second function called `or_query` that works in the same way as `and_query` you just implemented, but returns documents that contain _at least one_ of the words in the query. Demonstrate the working of this second function also with an example (again, choose one that leads to fewer than 100 hits).

In [ ]:
# Add your code here

### Task 3

Show how many hits the query "to be or not to be" returns for your two query functions (`and_query` and `or_query`).

In [ ]:
# Add your code here

### Task 4

Given the nature of our dataset, how many of the documents from task 3 do you think are actually about the [Shakespeare quote](https://en.wikipedia.org/wiki/To_be%2C_or_not_to_be)? What could you do to better handle such queries? (You don't have to implement this yet!)

**Answer:** [Write your answer text here]

### Task 5

Why does `and_query('moringa seed extracts')` not return our example paper 28792522, even though it mentions moringa seed extracts in the abstract? (You do not have to implement anything to fix this yet!)

**Answer:** [Write your answer text here]

## Submission

Submit the answers to the assignment as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers via Canvas. Don't forget to add your name, and remember that the assignments have to be done individually and group submissions are **not allowed**.